In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import pandas as pd
import numpy as np
import re
import matplotlib.pyplot as plt
from matplotlib import rcParams
import seaborn as sns
from datetime import datetime
import math, statistics

from sklearn.preprocessing import QuantileTransformer, OrdinalEncoder
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import accuracy_score, classification_report
from sklearn.svm import SVC
from xgboost import XGBClassifier

In [ ]:
df = pd.read_csv("/content/drive/MyDrive/df_for_unstructure.csv", index_col = 0)

/usr/local/lib/python3.7/dist-packages/numpy/lib/arraysetops.py:580: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  mask |= (ar1 == a)


In [ ]:
y_pred_proba = pd.read_csv('/content/drive/MyDrive/y_pred_proba.csv', index_col=0)

In [ ]:
df = df[['gamer_id','game_id','login_total','login_mean','cluster','y']]

In [ ]:
df = df.drop_duplicates(subset='gamer_id', keep='first')

In [ ]:
df = df.reset_index()

In [ ]:
df = pd.concat([df,y_pred_proba],axis=1).drop(columns='index')

In [ ]:
# high cardinality 특성 확인
len(set(df['game_id']))

397

In [ ]:
# 레이블 균형 확인
df['y'].value_counts(normalize=True)

1    0.578954
0    0.421046
Name: y, dtype: float64

In [ ]:
# 데이터 분할
X = df[['game_id', 'login_total', 'login_mean', 'cluster', 'y_pred_proba']]
y = df['y']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.2, random_state=42)

print(X_train.shape)
print(X_val.shape)
print(X_test.shape)
print(y_train.shape)
print(y_val.shape)
print(y_test.shape)

(349383, 5)
(87346, 5)
(109183, 5)
(349383,)
(87346,)
(109183,)


# 모델링

In [ ]:
# 인코딩 X ==> game_id의 경우 train데이터에는 존재하지만 test데이터에는 존재하지 않는 unknown value 존재
"""encoder = OrdinalEncoder(handle_unknown = 'use_encoded_value', unknown_value=999)
X_train_encoded = encoder.fit_transform(X_train) # 학습데이터
X_test_encoded = encoder.transform(X_test) # 테스트데이터"""

In [ ]:
# Scaler : QuantileTransformer 

scaler = QuantileTransformer()
scaler.fit(X_train)
X_train_scaled = scaler.transform(X_train)
X_val_scaled = scaler.transform(X_val)
X_test_scaled = scaler.transform(X_test)

In [ ]:
X_train_scaled

array([[0.56656657, 0.        , 0.        , 0.        , 0.77398093],
       [0.56656657, 0.        , 0.        , 1.        , 0.52952953],
       [0.56656657, 0.        , 0.        , 1.        , 0.34034034],
       ...,
       [0.56656657, 0.66466466, 0.99349349, 1.        , 0.30028155],
       [0.72472472, 0.        , 0.95195195, 0.48948949, 0.26806716],
       [0.03403403, 0.        , 0.75975976, 0.48948949, 0.10719551]])

In [ ]:
# 모델링

XGB_clf = XGBClassifier(
    n_estimators=1000,  # <= 1000 트리로 설정했지만, early stopping 에 따라 조절
    max_depth=5,        # default=3, high cardinality 특성이 존재할 경우 높임(game_id)
    learning_rate=0.1,
    n_jobs=-1,
    random_state=2
#   scale_pos_weight=ratio, # imbalance 데이터 일 경우 비율을 적용    
)

eval_set = [(X_train_scaled, y_train),
            (X_val_scaled, y_val)]

# 학습

In [ ]:
XGB_clf.fit(X_train_scaled, y_train, 
          eval_set=eval_set,
          eval_metric='error', # #(wrong cases)/#(all cases)
          early_stopping_rounds=50
         ) # 50 rounds 동안 스코어의 개선이 없으면 멈춤

[0]	validation_0-error:0.084495	validation_1-error:0.086175
Multiple eval metrics have been passed: 'validation_1-error' will be used for early stopping.

Will train until validation_1-error hasn't improved in 50 rounds.
[1]	validation_0-error:0.080717	validation_1-error:0.082076
[2]	validation_0-error:0.079981	validation_1-error:0.08124
[3]	validation_0-error:0.08051	validation_1-error:0.081778
[4]	validation_0-error:0.078979	validation_1-error:0.080141
[5]	validation_0-error:0.078979	validation_1-error:0.080141
[6]	validation_0-error:0.077955	validation_1-error:0.079122
[7]	validation_0-error:0.077957	validation_1-error:0.079122
[8]	validation_0-error:0.077766	validation_1-error:0.078985
[9]	validation_0-error:0.076449	validation_1-error:0.077428
[10]	validation_0-error:0.075078	validation_1-error:0.076558
[11]	validation_0-error:0.075327	validation_1-error:0.076809
[12]	validation_0-error:0.074571	validation_1-error:0.075779
[13]	validation_0-error:0.074726	validation_1-error:0.0761

XGBClassifier(max_depth=5, n_estimators=1000, n_jobs=-1, random_state=2)

# 테스트 검증

In [ ]:
results = XGB_clf.evals_result()
train_error = results['validation_0']['error']
val_error = results['validation_1']['error']

epoch = range(1, len(train_error)+1)
plt.plot(epoch, train_error, label='Train')
plt.plot(epoch, val_error, label='Validation')
plt.ylabel('Classification Error')
plt.xlabel('Model Complexity (n_estimators)')
plt.ylim((0.05, 0.09)) # Zoom in
plt.legend();

# 학습데이터 : 트리 만들수록 성능 향상
# 검증데이터 : 트리 35개 부근부터 성능 변화 없음 (early stopping)

In [ ]:
print('테스트 정확도', XGB_clf.score(X_test_scaled, y_test))
print(classification_report(y_test, XGB_clf.predict(X_test_scaled)))

테스트 정확도 0.9417949680811115
              precision    recall  f1-score   support

           0       0.94      0.92      0.93     45828
           1       0.94      0.96      0.95     63355

    accuracy                           0.94    109183
   macro avg       0.94      0.94      0.94    109183
weighted avg       0.94      0.94      0.94    109183

